In [48]:
from pathlib import Path
import json

annotations = []
for p in Path(".").glob("*.json"):
    with open(p, encoding="utf-8") as f:
        print(p)
        annotations.append(json.load(f))

evaluations = {
    sentence: [
        annotation[sentence] for annotation in annotations if sentence in annotation
    ]
    for sentence in {
        sentence for annotation in annotations for sentence in annotation.keys()
    }
}

X = [s for s in evaluations.keys()]
y = [int(sum(e) > 0) for e in evaluations.values()]

# !pip install transformers datasets great-ai==0.0.6

evaluation-experiment-2-stage #1-2m6dmb.json
evaluation-experiment-2-stage #1-sa6a0y.json
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement great-ai==0.0.6 (from versions: none)
ERROR: No matching distribution found for great-ai==0.0.6


In [22]:
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback,
)
import numpy as np
from datasets import Dataset, load_metric

MODEL = "allenai/scibert_scivocab_uncased"
BATCH_SIZE = 32

tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=2)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


def tokenize_function(v):
    return tokenizer(v["text"])


dataset = (
    Dataset.from_dict({"text": X, "label": y})
    .map(lambda v: tokenizer(v["text"], truncation=True), batched=True)
    .remove_columns("text")
    .train_test_split(test_size=0.2, shuffle=True)
)

f1_score = load_metric("f1")


def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    return f1_score.compute(predictions=pred, references=labels)


training_args = TrainingArguments(
    output_dir=Path("models"),
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    save_total_limit=5,
    num_train_epochs=50,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    weight_decay=0.01,
    metric_for_best_model="f1",
    load_best_model_at_end=True,
)

result = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)],
).train()

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/allenai/scibert_scivocab_uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/858852fd2471ce39075378592ddc87f5a6551e64c6825d1b92c8dab9318e0fc3.03ff9e9f998b9a9d40647a2148a202e3fb3d568dc0f170dda9dda194bab4d5dd
Model config BertConfig {
  "_name_or_path": "allenai/scibert_scivocab_uncased",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 31090
}

loading file https://hug

  0%|          | 0/1 [00:00<?, ?ba/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 400
  Num Epochs = 50
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 650


Epoch,Training Loss,Validation Loss,F1
1,0.586800,0.512138,0.719101
2,0.411600,0.416675,0.849057
3,0.245600,0.417070,0.864000
4,0.147800,0.575878,0.852459
5,0.056800,0.474259,0.896552
6,0.022500,0.754236,0.843137
7,0.001000,0.857636,0.834783
8,0.000500,0.920232,0.869565
9,0.000300,0.970790,0.877193
10,0.000300,0.948689,0.862385


***** Running Evaluation *****
  Num examples = 100
  Batch size = 32
Saving model checkpoint to models/checkpoint-13
Configuration saved in models/checkpoint-13/config.json
Model weights saved in models/checkpoint-13/pytorch_model.bin
Deleting older checkpoint [models/checkpoint-91] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 100
  Batch size = 32
Saving model checkpoint to models/checkpoint-26
Configuration saved in models/checkpoint-26/config.json
Model weights saved in models/checkpoint-26/pytorch_model.bin
Deleting older checkpoint [models/checkpoint-117] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 100
  Batch size = 32
Saving model checkpoint to models/checkpoint-39
Configuration saved in models/checkpoint-39/config.json
Model weights saved in models/checkpoint-39/pytorch_model.bin
Deleting older checkpoint [models/checkpoint-130] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 100
  Ba

In [44]:
from great_ai.large_file import LargeFileS3

LargeFileS3.configure_credentials_from_file("config.ini")

model.save_pretrained("pretrained")
LargeFileS3("scibert-highlights").push("pretrained")

Configuration saved in pretrained/config.json
Model weights saved in pretrained/pytorch_model.bin


  adding: pretrained/ (stored 0%)
  adding: pretrained/config.json (deflated 49%)
  adding: pretrained/pytorch_model.bin (deflated 7%)
